In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import openpyxl
import time

import googlemaps
from tqdm import tqdm  # for progress bar

import folium
from folium.plugins import MarkerCluster


In [ ]:
df = pd.read_excel(r"Enter the solar battery storage manufacturers excel file path")
df.head()

In [ ]:
df.columns

In [ ]:
new_df = df [[ 'Company_Name','Company_Name_URL', 'Area', 'Category', 'Technology',
        'company_name', 'address', 'phone_no',
       'company_email', 'company_website', 'country',
       'component_types', 'storage_system', 'technology', 'Inverter',
       'power_rangekWp']]

new_df

In [ ]:
new_df.rename(columns = {'Company_Name':'general_company_name', 'Area':'state/province', 
                              'company_name':'company_name_(chinese)','power_rangekWp':'power_range_(kWp)'}, inplace = True)

new_df.columns

In [ ]:
new_df.info()

In [ ]:
missing_values = new_df.isnull().sum()
missing_values


In [ ]:
dropped_rows = new_df[new_df['address'].isnull()]

print(dropped_rows)

In [ ]:
# First, let's see how many null values we have in the address column
print("Number of null values in address column:", new_df['address'].isnull().sum())

# Drop rows where address is null and create a new clean dataframe
new_df = new_df.dropna(subset=['address'])

# Verify the rows were dropped
print("\nOriginal dataset shape:", df.shape)
print("Clean dataset shape:", new_df.shape)


In [11]:
# Convert list columns to strings before dropping duplicates
list_columns = ['Technology_List', 'Category_List']
for col in list_columns:
    if col in new_df.columns:
        new_df[col] = new_df[col].astype(str)

# Now drop duplicates
new_df = new_df.drop_duplicates()

In [12]:
# 5. Data transformation
# If `power_range_(kWp)` is numeric but stored as object due to inconsistent formats
new_df['power_range_(kWp)'] = pd.to_numeric(new_df['power_range_(kWp)'], errors='coerce')


In [13]:
# Ensure URLs and emails are valid (you can use regex for a basic check)

new_df['company_website'] = new_df['company_website'].apply(lambda x: x if pd.notnull(x) and x.startswith('http') else 'Unknown')
new_df['company_email'] = new_df['company_email'].apply(lambda x: x if pd.notnull(x) and '@' in x else 'Unknown')


In [ ]:
new_df

In [ ]:
new_df.columns

In [ ]:
summary_stats = new_df.describe(include='all')
summary_stats

In [17]:
companies_by_state = new_df['state/province'].value_counts().reset_index()

In [ ]:
companies_by_state.columns = ['State/Province', 'Number of Companies']

companies_by_state

In [ ]:
# Set the style and palette
sns.set_theme(style="whitegrid")
palette = sns.color_palette("viridis", len(companies_by_state))



# Plotting the results
plt.figure(figsize=(24, 20))
ax = sns.barplot(
    y='State/Province', 
    x='Number of Companies', 
    data=companies_by_state, 
    palette=palette
)

# Add titles and labels
plt.title('Number of Companies by State/Province', fontsize=16, weight='bold')
plt.xlabel('Number of Companies', fontsize=14)
plt.ylabel('State/Province', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Add data labels to each bar
for index, value in enumerate(companies_by_state['Number of Companies']):
    ax.text(value, index, f'{value}', color='black', va="center", ha="left", fontsize=12)

# Remove top and right spines for cleaner look
sns.despine(left=True, bottom=True)

plt.show()

In [ ]:
# Missing value heatmap
plt.figure(figsize=(24, 16))
sns.heatmap(new_df.isnull(), cbar=False, cmap='viridis', yticklabels=False)
plt.title('Heatmap of Missing Values')
plt.show()

In [ ]:
# Count plot for `Category`

plt.figure(figsize=(20, 12))
sns.countplot(y='Category', data=new_df, order=new_df['Category'].value_counts().index)
plt.title('Distribution of Categories')
plt.xlabel('Count')
plt.ylabel('Category')
plt.show()

In [ ]:
# Ensure 'Technology' column is string type
new_df['Technology'] = new_df['Technology'].astype(str)

# Split the 'Technology' column into a list
new_df['Technology_List'] = new_df['Technology'].str.split(',')

# Explode the 'Technology_List' into separate rows
df_exploded = new_df.explode('Technology_List')

# Remove leading and trailing whitespace
df_exploded['Technology_List'] = df_exploded['Technology_List'].str.strip()

# Count the occurrences of each technology
technology_counts = df_exploded['Technology_List'].value_counts().reset_index()

# Rename columns for clarity
technology_counts.columns = ['Technology', 'Number of Companies']

# Display the result
print(technology_counts)

In [ ]:
# Sort the technologies for better visualization
technology_counts = technology_counts.sort_values(by='Number of Companies', ascending=False)

# Set the style
sns.set(style="whitegrid")

# Create a bar plot
plt.figure(figsize=(14, 8))
bar_plot = sns.barplot(
    x='Number of Companies', 
    y='Technology', 
    data=technology_counts,
    palette='coolwarm'
)

# Add title and labels
bar_plot.set_title('Number of Companies by Technology', fontsize=20, weight='bold')
bar_plot.set_xlabel('Number of Companies', fontsize=14, weight='bold')
bar_plot.set_ylabel('Technology', fontsize=14, weight='bold')

# Add counts on the bars
for index, value in enumerate(technology_counts['Number of Companies']):
    bar_plot.text(value, index, f' {value}', color='black', ha="left", fontsize=12)

# Improve layout and add gridlines
plt.tight_layout()
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Show the plot
plt.show()

In [ ]:
# Replace NaNs and empty strings with a placeholder
new_df['state/province'] = new_df['state/province'].replace('', 'Unknown State/Province').fillna('Unknown State/Province')
new_df['Category'] = new_df['Category'].replace('', 'Unknown Category').fillna('Unknown Category')

# Split the 'Category' column into a list of unique categories
new_df['Category_List'] = new_df['Category'].str.split(',')

# Explode the 'Category_List' into separate rows
df_exploded = new_df.explode('Category_List')

# Remove leading and trailing whitespace
df_exploded['Category_List'] = df_exploded['Category_List'].str.strip()

# Add a count column (optional, useful for the sunburst chart)
df_exploded['Count'] = 1

# Plot Sunburst Chart
fig = px.sunburst(
    df_exploded,
    path=['state/province', 'Category_List'],  # Use 'state/province' instead of 'country'
    values='Count',  # Use 'Count' for the size of the sectors
    title='Company Distribution by State/Province and Unique Categories',
    hover_data={'Count': True}  # Display 'Count' in hover data
)

# Update layout for better aesthetics
fig.update_layout(
    title_font_size=24,
    title_x=0.5,
    uniformtext=dict(minsize=12, mode='hide'),  # Hide text for small sectors
    width=1200,  # Make the chart larger
    height=900  # Make the chart larger
)

fig.show()

In [ ]:
# Prepare data for the Sankey diagram
sankey_data = new_df.groupby(['state/province', 'Category']).size().reset_index(name='Count')

# Create unique lists for source and target
states = list(sankey_data['state/province'].unique())
categories = list(sankey_data['Category'].unique())

# Create index mappings for the states and categories
state_indices = {state: i for i, state in enumerate(states)}
category_indices = {category: i + len(states) for i, category in enumerate(categories)}

# Map source and target to their respective indices
source_indices = sankey_data['state/province'].map(state_indices)
target_indices = sankey_data['Category'].map(category_indices)

# Colors for nodes
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']
node_colors = [colors[i % len(colors)] for i in range(len(states) + len(categories))]

# Create the Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=30,
        line=dict(color="black", width=0.5),
        label=states + categories,
        color=node_colors
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=sankey_data['Count'],
        color=[f'rgba(63, 81, 181, {0.5 + 0.1 * (v / max(sankey_data["Count"]))})' for v in sankey_data['Count']] # Gradient color based on value
    )
))

# Update layout
fig.update_layout(
    title_text="Sankey Diagram from State/Province to Category",
    font_size=12,
    width=1000,
    height=1000
)

fig.show()

In [ ]:
# First, fill any NaN values with a placeholder
# First create the locat DataFrame from new_df
locat = new_df[['general_company_name', 'address', 'state/province', 'country']].copy()

# Fill any NaN values with a placeholder
locat['country'] = locat['country'].fillna('Unknown')

# Convert all columns to string type
locat['address'] = locat['address'].astype(str)
locat['general_company_name'] = locat['general_company_name'].astype(str)
locat['state/province'] = locat['state/province'].astype(str)
locat['country'] = locat['country'].astype(str)

# Now create the full_address
locat['full_address'] = locat[['general_company_name', 'address', 'state/province', 'country']].apply(lambda x: ','.join(x), axis=1)

# Print the first few rows to verify 
print(locat.head())

# Print shape to see how many records we have
print("\nShape of the dataset:", locat.shape)

In [27]:
# locat['full_address'] = locat[['general_company_name','address', 'state/province', 'country']].apply(lambda x: '-'.join(x), axis=1)
# locat

In [ ]:
locat['full_address'].head()

In [ ]:
locat.columns

In [ ]:
# Install required package
#!pip install googlemaps



def get_coordinates_google(address, api_key):
    """
    Get coordinates using Google Maps Geocoding API
    """
    gmaps = googlemaps.Client(key=api_key)
    
    try:
        # Add delay to respect rate limits
        time.sleep(0.1)
        
        # Geocode the address
        result = gmaps.geocode(address)
        
        if result:
            location = result[0]['geometry']['location']
            return (location['lat'], location['lng'])
        return None
        
    except Exception as e:
        print(f"Error with address: {address}")
        print(f"Error message: {str(e)}")
        return None

def batch_geocode(df, api_key):
    """
    Batch process addresses with progress bar
    """
    # Initialize empty lists for coordinates
    lats = []
    lngs = []
    
    # Create progress bar
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Geocoding addresses"):
        # Combine address components
        address = f"{row['general_company_name']},{row['address']}, {row['state/province']}, {row['country']}"
        
        # Get coordinates
        coords = get_coordinates_google(address, api_key)
        
        if coords:
            lats.append(coords[0])
            lngs.append(coords[1])
        else:
            lats.append(None)
            lngs.append(None)
    
    return lats, lngs

# Your Google Maps API key
GOOGLE_API_KEY = 'Enter google maps api key (geocoding API)'  # Replace with your API key

# Process the addresses
print("Starting geocoding process...")
latitudes, longitudes = batch_geocode(locat, GOOGLE_API_KEY)

# Add coordinates to dataframe
locat['latitude'] = latitudes
locat['longitude'] = longitudes

# Save results to CSV
locat.to_csv('geocoded_locations_google.csv', index=False)

# Print statistics
success_rate = (locat['latitude'].notna().sum() / len(locat)) * 100
print(f"\nGeocoding success rate: {success_rate:.2f}%")

# Display first few results
print("\nFirst few results:")
print(locat[['address', 'latitude', 'longitude']].head())

In [32]:

# Create a map centered on China
m = folium.Map(location=[35.8617, 104.1954], zoom_start=4)

# Add a marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Add markers for each location
for idx, row in locat.iterrows():
    if pd.notna(row['latitude']) and pd.notna(row['longitude']):
        folium.Marker(
            [row['latitude'], row['longitude']],
            popup=f"<b>{row['general_company_name']}</b><br>{row['address']}",
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(marker_cluster)

# Save the map
m.save('china_companies_map.html')